In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [2]:
# Read the Excel file
df_layoff_pd = pd.read_excel('tech_layoffs.xlsx')

# Read the TSV file
df_housing_pd = pd.read_csv("latest_weekly_housing_market_data_most_recent.tsv000", sep='\t')

In [3]:
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions, QueryOptions
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions, QueryOptions
from couchbase.bucket import Bucket
from couchbase.management.buckets import CreateBucketSettings

In [4]:
import os
import dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
couchbaseUsername, couchbasePassword = os.getenv('COUCHBASE_USERNAME'), os.getenv('COUCHBASE_PASSWORD')


# get a reference to our cluster
auth = PasswordAuthenticator(couchbaseUsername, couchbasePassword)
cluster = Cluster.connect('couchbase://127.0.0.1', ClusterOptions(auth))

In [10]:
def db_populator(datasets, options=None):
    for bucketName, dataset in datasets:
        if options is not None:
            bucketSettings = CreateBucketSettings(name=options["name"], 
                                                  bucket_type=options["bucket_type"], 
                                                  ram_quota_mb=options["ram_quota_mb"], 
                                                  max_expiry=options["max_expiry"], 
                                                  compression_mode=options["compression_mode"], 
                                                  conflict_resolution_type=options["conflict_resolution_type"])
        else:
            bucketSettings = CreateBucketSettings(name=bucketName, 
                                                  bucket_type="couchbase", 
                                                  ram_quota_mb=100, 
                                                  max_expiry=0, 
                                                  compression_mode="passive", 
                                                  conflict_resolution_type="seqno")
            
        print(f"Populating {bucketName} with {len(dataset)} documents")

        try: 
            cluster.buckets().create_bucket(bucketSettings)
            print('Bucket created')
            cluster.wait_until_ready(20)
        except Exception as e:
            if "Bucket with given name already exists" in str(e):
                pass

        bucket = cluster.bucket(bucketName)
        collection = bucket.default_collection()

        i = 0
        for _, row in dataset.iterrows():
            data = row.to_dict()

            # Convert non-serializable objects (e.g., Timestamps) to strings or appropriate formats
            for key, value in data.items():
                if isinstance(value, pd.Timestamp):
                    data[key] = value.isoformat()  # or use value.timestamp() for Unix timestamp

            data["_id"] = str(i)
            collection.upsert(str(i), data)
            i += 1

        print('Data stored')

        try:
            cluster.query_indexes().create_primary_index(bucketName)
            print('Index created')
        except Exception as e:
            if "already exists" in str(e):
                pass


In [11]:
datasets = [("tech_layoffs", df_layoff_pd), ("housing_market", df_housing_pd)]

In [12]:
db_populator(datasets)

Populating tech_layoffs with 1418 documents
Data stored
Populating housing_market with 4246231 documents
Bucket created


TemporaryFailException: TemporaryFailException(<ec=13, category=couchbase.common, message=ambiguous_timeout (13), context=KeyValueErrorContext:{'retry_attempts': 7, 'retry_reasons': {'key_value_temporary_failure'}, 'key': '1258651', 'bucket_name': 'housing_market', 'scope_name': '_default', 'collection_name': '_default', 'opaque': 1258907}, C Source=/Users/couchbase/jenkins/workspace/python/sdk/python-packaging-pipeline/py-client/src/kv_ops.cxx:651>)